# Produce PDF report of network stations showing RF waveforms

In [ ]:
# Imports
import os
# from collections import defaultdict

import numpy as np
import rf
import rf.imaging

In [ ]:
import importlib
import seismic.receiver_fn.rf_util as rf_util
import seismic.receiver_fn.rf_plot_utils as rf_plot_utils
import seismic.receiver_fn.rf_stacking as rf_stacking

In [ ]:
# importlib.reload(rf_plot_utils)
# importlib.reload(rf_util)

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

## Read source file

In [ ]:
rf_type = 'ZRT_td'

In [ ]:
network = 'OA'
# network = '7X'

In [ ]:
src_file = r"..\DATA\OA_rfs_20170911T000036-20181128T230620_{}_rev9_qual.h5".format(rf_type)
# src_file = r"..\DATA\7X_rfs_20090616T034200-20110401T231849_{}_rev1_qual.h5".format(rf_type)

In [ ]:
data_all = rf_util.read_h5_rf(src_file)

In [ ]:
# data_all[0].stats

## Convert to hierarchical dictionary format

In [ ]:
data_dict = rf_util.rf_to_dict(data_all)

## Load event mask files if available

In [ ]:
event_mask_path = r"C:\software\hiperseis\seismic\receiver_fn\DATA\event_mask"

In [ ]:
event_mask_dict = None
if event_mask_path is not None:
    import re
    mask_files = os.listdir(event_mask_path)
    mask_files = [f for f in mask_files if os.path.isfile(os.path.join(event_mask_path, f))]
#     print(mask_files)
    pattern = "([A-Za-z0-9\.]{5,})_event_mask\.txt"
    pattern = re.compile(pattern)
    event_mask_dict = dict()
    for f in mask_files:
        match_result = pattern.match(f)
        if not match_result:
            continue
        code = match_result[1]
#         print(code)
        with open(os.path.join(event_mask_path, f), 'r') as f:
            events = f.readlines()
            events = [e.strip() for e in events]
            event_mask_dict[code] = events
        # end with
    # end for
# end if

## Plot a few items interactively to check data

In [ ]:
i = 0
for st, station_db in data_dict.items():
    i += 1
    if i > 3:
        break
    channel = rf_util.choose_rf_source_channel(rf_type, station_db)
    channel_data = station_db[channel]
    full_code = '.'.join([network, st, channel])

    t_channel = list(channel)
    t_channel[-1] = 'T'
    t_channel = ''.join(t_channel)

    if event_mask_dict and full_code in event_mask_dict:
        # Select events from external source
        event_mask = event_mask_dict[full_code]
        rf_stream = rf.RFStream([tr for tr in channel_data if tr.stats.event_id in event_mask]).sort(['back_azimuth'])
    else:
        rf_util.label_rf_quality_simple_amplitude(rf_type, channel_data)
        rf_stream = rf.RFStream([tr for tr in channel_data if tr.stats.predicted_quality == 'a']).sort(['back_azimuth'])
    # end if

    # Find matching HHT channel data
    events = [tr.stats.event_id for tr in rf_stream]
    transverse_data = station_db[t_channel]
    t_stream = rf.RFStream([tr for tr in transverse_data if tr.stats.event_id in events]).sort(['back_azimuth'])

    # Plot to notebook
    fig = rf_plot_utils.plot_rf_wheel([rf_stream, t_stream], layout=(1,2))
    plt.show()
    
    fig = rf_plot_utils.plot_rf_stack(rf_stream)
    plt.show()
    
    fig = rf_plot_utils.plot_rf_stack(t_stream)
    plt.show()

## Plot all data to PDF file

In [ ]:
with PdfPages('{}_RF_summary_handpicked.pdf'.format(network)) as pdf:
    plt.rc('text', usetex=False)  # Would like to use Tex, but lack desktop PC privileges to update packages to what is required
    for st, station_db in data_dict.items():
        # Choose RF channel
        channel = rf_util.choose_rf_source_channel(rf_type, station_db)
        channel_data = station_db[channel]
        full_code = '.'.join([network, st, channel])

        t_channel = list(channel)
        t_channel[-1] = 'T'
        t_channel = ''.join(t_channel)

        if event_mask_dict and full_code in event_mask_dict:
            # Select events from external source
            event_mask = event_mask_dict[full_code]
            rf_stream = rf.RFStream([tr for tr in channel_data if tr.stats.event_id in event_mask]).sort(['back_azimuth'])
        else:
            # Label and filter quality
            rf_util.label_rf_quality_simple_amplitude(rf_type, channel_data)
            rf_stream = rf.RFStream([tr for tr in channel_data if tr.stats.predicted_quality == 'a']).sort(['back_azimuth'])
        # end if
        if not rf_stream:
            continue

        # Find matching HHT channel data
        events = [tr.stats.event_id for tr in rf_stream]
        transverse_data = station_db[t_channel]
        t_stream = rf.RFStream([tr for tr in transverse_data if tr.stats.event_id in events]).sort(['back_azimuth'])
        if not t_stream:
            continue

        # Plot pinwheel of primary and transverse components
        fig = rf_plot_utils.plot_rf_wheel([rf_stream, t_stream], fontscaling=0.8)
        fig.set_size_inches(8.27, 11.69)
        plt.tight_layout()
        plt.subplots_adjust(hspace=0.15, top=0.95, bottom=0.15)
        ax = fig.gca()
        fig.text(-0.32, -0.32, "\n".join(rf_stream[0].stats.processing), fontsize=6, transform=ax.transAxes)
        pdf.savefig(dpi=300, papertype = 'a4', orientation = 'portrait')
        plt.close()

        # Plot RF stack of primary component
        fig = rf_plot_utils.plot_rf_stack(rf_stream)
        fig.set_size_inches(8.27, 11.69)
        fig.suptitle("Channel {}".format(rf_stream[0].stats.channel))
        pdf.savefig(dpi=300, papertype = 'a4', orientation = 'portrait')
        plt.close()

        # Plot RF stack of transverse component
        fig = rf_plot_utils.plot_rf_stack(t_stream)
        fig.set_size_inches(8.27, 11.69)
        fig.suptitle("Channel {}".format(t_stream[0].stats.channel))
        pdf.savefig(dpi=300, papertype = 'a4', orientation = 'portrait')
        plt.close()
    # end for
# end with